# Primitive Roots and Generators

**Module 05** | 05-discrete-log-diffie-hellman

*primitive_root(), generator choice*

## Objectives

By the end of this notebook you will be able to:

1. Understand the core ideas behind **primitive roots and generators**.
2. Explore these concepts interactively using SageMath.
3. Build intuition through hands-on computation and visualization.

## Prerequisites

- Completion of [The Discrete Logarithm Problem](05a-the-discrete-log-problem.ipynb).
- Concepts and notation introduced in the previous notebook.

## Primitive Roots

In [ ]:
# A primitive root of p generates all of (Z/pZ)*
p = 23
g = primitive_root(p)
print(f'Primitive root of {p}: {g}')

## Verifying a Generator

In [ ]:
# g is a generator iff its order equals p-1
p = 23
g = Mod(5, p)
print(f'Order of {g}: {g.multiplicative_order()}')
print(f'Is generator: {g.multiplicative_order() == p-1}')

## Finding All Generators

In [ ]:
# List all primitive roots of a prime
p = 23
gens = [g for g in range(1,p) if Mod(g,p).multiplicative_order() == p-1]
print(f'Generators of (Z/{p}Z)*: {gens}')

## Exercises

Try these on your own before moving on:

1. **Exercise 1:** *(TODO: add exercise)*
2. **Exercise 2:** *(TODO: add exercise)*
3. **Exercise 3:** *(TODO: add exercise)*

## Summary

In this notebook we explored **primitive roots and generators**.  Key takeaways:

- *(TODO: summarize key point 1)*
- *(TODO: summarize key point 2)*
- *(TODO: summarize key point 3)*

**Next:** [Diffie-Hellman Key Exchange](05c-diffie-hellman-key-exchange.ipynb)